In [13]:
import pandas as pd
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

In [4]:
main_df = pd.read_csv('./datas/feature_extracted_dataset.csv') 
print(f"Öznitelik seti yüklendi. Boyut: {main_df.shape}")

Öznitelik seti yüklendi. Boyut: (73995, 15)


In [14]:
required_nltk_resources = {
    'stopwords': ('corpora/stopwords', False), # (kaynak_yolu, force_download_flag)
    'punkt': ('tokenizers/punkt', True)     # Punkt için force=True deneyelim
}

all_resources_available = True
for resource_name, (resource_path, force_download) in required_nltk_resources.items():
    try:
        nltk.data.find(resource_path)
        print(f"NLTK kaynağı '{resource_name}' zaten mevcut.")
    except LookupError:
        print(f"NLTK kaynağı '{resource_name}' bulunamadı, indiriliyor (force={force_download})...")
        try:
            nltk.download(resource_name, force=force_download, quiet=True)
            nltk.data.find(resource_path)
            print(f"NLTK kaynağı '{resource_name}' başarıyla indirildi ve bulundu.")
        except Exception as e:
            print(f"NLTK kaynağı '{resource_name}' indirilirken/bulunurken bir hata oluştu: {e}")
            all_resources_available = False
            break

NLTK kaynağı 'stopwords' zaten mevcut.
NLTK kaynağı 'punkt' zaten mevcut.


In [15]:
def preprocess_text_with_stemmer_lang(text):
            if not isinstance(text, str):
                return ""
            
            text = text.lower()
            text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
            text = re.sub(r'\@\w+', '', text)
            text = re.sub(r'#\w+', '', text) 
            text = re.sub(r'[^a-z\s]', '', text)
            
            # Cümlelere ayırırken ve kelimelere ayırırken dili belirtelim
            # Eğer metin zaten çok kısaysa veya tek bir kelimeyse sent_tokenize sorun çıkarabilir,
            # bu yüzden doğrudan word_tokenize da bir seçenek olabilir veya try-except eklenebilir.
            # Şimdilik basit tutalım:
            try:
                # sent_tokenize boş string için sorun çıkarmaz, boş liste döndürür.
                sentences = sent_tokenize(text, language='english')
                tokens = []
                for sentence in sentences:
                    tokens.extend(word_tokenize(sentence, language='english'))
            except Exception as e_tok: # Tokenizasyon sırasında beklenmedik bir hata olursa
                # print(f"Tokenizasyon hatası: {e_tok} - Metin: '{text[:50]}...'") # Hata ayıklama için
                tokens = text.split() # Basit bir fallback

            processed_tokens = [stemmer_obj.stem(word) for word in tokens if word not in stop_words_list and len(word) > 2]
            
            return " ".join(processed_tokens)

In [17]:
from nltk.corpus import stopwords

stop_words_list = set(stopwords.words('english'))
stemmer_obj = PorterStemmer()

main_df['processed_text'] = main_df['tweet'].astype(str).apply(preprocess_text_with_stemmer_lang)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000, min_df=5, ngram_range=(1,1))
tfidf_features = tfidf_vectorizer.fit_transform(main_df['processed_text'])
print(f"TF-IDF boyut: {tfidf_features.shape}")

TF-IDF boyut: (73995, 5000)


In [24]:
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import numpy as np

# 1) TF-IDF Vektörleri
tfidf_vectorizer = TfidfVectorizer(max_features=5000, min_df=5, ngram_range=(1,1))
tfidf_features = tfidf_vectorizer.fit_transform(main_df['processed_text'])
print(f"TF-IDF boyut: {tfidf_features.shape}")

# 2) Explicit entity özniteliği
explicit_entity = main_df['explicitly_contains_entity']\
    .fillna(False).astype(int)\
    .values.reshape(-1, 1)
print(f"Explicit entity boyut: {explicit_entity.shape}")

# 3) Numerik öznitelikler
num_cols = ['ctx_neg','ctx_neu','ctx_pos','ctx_compound','emb_sim','fuzzy_max']
numeric_array = main_df[num_cols].fillna(0).values
numeric_sparse = sparse.csr_matrix(numeric_array)
print(f"Numerik öznitelikler boyut: {numeric_sparse.shape}")

# 4) Tüm öznitelikleri birleştirme
all_features = hstack([
    tfidf_features,
    explicit_entity,
    numeric_sparse
])
print(f"Nihai öznitelik matrisi boyut: {all_features.shape}")

# 5) Hedef değişken
Y = main_df['Sentiment']
print(f"Hedef değişken boyut: {Y.shape}")


TF-IDF boyut: (73995, 5000)
Explicit entity boyut: (73995, 1)
Numerik öznitelikler boyut: (73995, 6)
Nihai öznitelik matrisi boyut: (73995, 5007)
Hedef değişken boyut: (73995,)


In [27]:
import joblib
from scipy.sparse import save_npz
import os

# --- Kaydedilecek Dizinlerin Oluşturulması ---
# Bu dizinlerin projenizde var olduğundan emin olun, yoksa oluşturun.
os.makedirs('./saved_objects/', exist_ok=True)
os.makedirs('./datas/', exist_ok=True)
print("Kaydedilecek dizinler kontrol edildi/oluşturuldu.")

# --- Dosya Yollarının Tanımlanması ---
TFIDF_VECTORIZER_PATH = './saved_objects/tfidf_vectorizer.joblib'
TFIDF_FEATURES_PATH = './saved_objects/tfidf_features.npz'
MANUAL_FEATURES_PATH = './datas/manuel_feature_extracted.csv'

print("\n>>> Oluşturulan Önemli Nesneler Diske Kaydediliyor...")

try:
    # 1. TF-IDF Vektörleyicisini Kaydetme
    # Bu, test verisini aynı kelime dağarcığı ile dönüştürmek için kritik öneme sahiptir.
    joblib.dump(tfidf_vectorizer, TFIDF_VECTORIZER_PATH)
    print(f"- TfidfVectorizer başarıyla '{TFIDF_VECTORIZER_PATH}' dosyasına kaydedildi.")

    # 2. TF-IDF Öznitelik Matrisini Kaydetme
    # Bu büyük matrisi her seferinde yeniden hesaplamamak için kaydediyoruz.
    save_npz(TFIDF_FEATURES_PATH, tfidf_features)
    print(f"- TF-IDF sparse matrisi başarıyla '{TFIDF_FEATURES_PATH}' dosyasına kaydedildi.")

    # 3. Manuel Öznitelikleri ve Hedef Değişkeni İçeren DataFrame'i Kaydetme
    # Bu CSV, test betiğimizin ana veri kaynağı olacak.
    # Sadece gerekli sütunları seçerek daha temiz bir dosya oluşturalım.
    cols_to_save = [
        'Sentiment', 
        'processed_text', 
        'explicitly_contains_entity', 
        'ctx_neg', 'ctx_neu', 'ctx_pos', 'ctx_compound', 
        'emb_sim', 
        'fuzzy_max'
    ]
    # main_df'te bu sütunların olduğundan emin olalım
    if all(col in main_df.columns for col in cols_to_save):
        main_df[cols_to_save].to_csv(MANUAL_FEATURES_PATH, index=False)
        print(f"- Manuel öznitelikler ve hedef değişken başarıyla '{MANUAL_FEATURES_PATH}' dosyasına kaydedildi.")
    else:
        print(f"HATA: Gerekli sütunlar 'main_df' içinde bulunamadı. CSV dosyası oluşturulamadı.")

    print("\n>>> Kaydetme işlemi başarıyla tamamlandı!")

except Exception as e:
    print(f"\nKaydetme sırasında bir hata oluştu: {e}")



Kaydedilecek dizinler kontrol edildi/oluşturuldu.

>>> Oluşturulan Önemli Nesneler Diske Kaydediliyor...
- TfidfVectorizer başarıyla './saved_objects/tfidf_vectorizer.joblib' dosyasına kaydedildi.
- TF-IDF sparse matrisi başarıyla './saved_objects/tfidf_features.npz' dosyasına kaydedildi.
- Manuel öznitelikler ve hedef değişken başarıyla './datas/manuel_feature_extracted.csv' dosyasına kaydedildi.

>>> Kaydetme işlemi başarıyla tamamlandı!


In [28]:
import pandas as pd

# 0) İkincil veri setinizi okuyun (yolunuzu güncelleyin)
extra_df = pd.read_csv("./datas/secondary_dataset.csv", encoding="utf8")

# 1) processed_text sütununu main_df ile aynı fonksiyonla oluşturun
#    Aşağıda önceki hücrenizdeki preprocess_text_with_stemmer_lang fonksiyonunu kullanıyoruz.
extra_df['processed_text'] = extra_df['text'].astype(str).apply(preprocess_text_with_stemmer_lang)

# 2) Artık pipe_extra’yu eğitebilirsiniz:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_extra = make_pipeline(
    TfidfVectorizer(max_features=3000, ngram_range=(1,2)),
    LogisticRegression(max_iter=200, random_state=42)
)
pipe_extra.fit(extra_df['processed_text'], extra_df['label'])

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=3000, ngram_range=(1, 2))),
                ('logisticregression',
                 LogisticRegression(max_iter=200, random_state=42))])

In [29]:
# Eğitim notebook'unuzda bu kodu çalıştırın
import joblib

# pipe_extra'yı eğittiğiniz hücreden sonra bu hücreyi ekleyin
joblib.dump(pipe_extra, "./saved_objects/pipe_extra_sentiment.joblib")

print("pipe_extra_sentiment.joblib dosyası başarıyla kaydedildi.")

pipe_extra_sentiment.joblib dosyası başarıyla kaydedildi.


In [30]:
import pandas as pd
import numpy as np
import joblib
import time
import matplotlib.pyplot as plt
import warnings

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score, f1_score
from scipy.sparse import hstack, csr_matrix, load_npz

In [31]:
# Test edilecek modellerin adları ve dosya isimleri
model_files = {
    # Normal Modeller
    'ExtraTrees_normal': 'best_extra_trees.pkl',
    'MLP_normal': 'best_mlp.pkl',
    'RandomForest_normal': 'best_rf.pkl',
    'XGBoost_normal': 'best_xgb_sklearn.pkl',
    'LinearSVC_normal': 'best_lsvc.pkl',
    'LogisticRegression_normal': 'best_logreg.pkl',
    'MultinomialNB_normal': 'best_nb.pkl',
    # Geliştirilmiş (Enhanced) Modeller
    'ExtraTrees_enhanced': 'et_enhanced.pkl',
    'MLP_enhanced': 'mlp_enhanced.pkl',
    'RandomForest_enhanced': 'rf_enhanced.pkl',
    'XGBoost_enhanced': 'xgb_enhanced_sklearn.pkl',
    'LinearSVC_enhanced': 'lsvc_enhanced.pkl',
    'LogisticRegression_enhanced': 'lr_enhanced.pkl',
    'MultinomialNB_enhanced': 'mnb_enhanced.pkl',
}